In [1]:
import sys
sys.path.append("/home/chaoqiy2/github/PyHealth-OMOP")

from warnings import simplefilter

# ignore all warnings
simplefilter(action='ignore')

# 1. load dataset

In [2]:
from pyhealth.datasets import MIMIC3BaseDataset

# used for development
# base_ds = MIMIC3BaseDataset(root="/srv/local/data/physionet.org/files/mimiciii/1.4", flag="dev")

# used for product
base_ds = MIMIC3BaseDataset(root="/srv/local/data/physionet.org/files/mimiciii/1.4", flag="prod")

# 2. drug recommendation task

In [3]:
from pyhealth.tasks import DrugRecDataset
drugrec_ds = DrugRecDataset(base_ds)

----- preparing code mappings -----
source loaded from ./data cache
source loaded from ./data cache
source loaded from ./data cache
source loaded from ./data cache
mapping finished: RxNorm -> ATC4
mapping finished: ATC4 -> RxNorm
load time: 2.75935697555542s
-----------------------------------------


100%|█████████████████████████████████████████████████████████████████████████| 46518/46518 [00:06<00:00, 7319.67it/s]


1. finish cleaning the dataset for drug recommendation task
2. tokenized the medical codes


In [4]:
from pyhealth.data.split import split_by_pat
train_loader, val_loader, test_loader = split_by_pat(
                                            drugrec_ds,
                                            ratios = [2/3, 1/6, 1/6], 
                                            batch_size = 64, 
                                            seed = 12345,
                                        )

1. finish data splitting
2. generate train / val / test data loaders


# 3. initialize DL models

In [22]:
from pyhealth.models import RETAIN, MLModel, RNN, Transformer, GAMENet, SafeDrug
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

# ------ DL model -------

# # for GAMENet
# visit_ls = train_loader.dataset.indices
# ehr_adj = train_loader.dataset.dataset.generate_ehr_adj_for_GAMENet(visit_ls)
# ddi_adj = train_loader.dataset.dataset.get_ddi_matrix()
# model = GAMENet(
#     task = "drug_recommendation",
#     voc_size = drugrec_ds.voc_size,
#     tokenizers = drugrec_ds.tokenizers,
#     emb_dim = 64,
#     ehr_adj = ehr_adj,
#     ddi_adj = ddi_adj,
# )

# for SafeDrug
# bipartite_info = train_loader.dataset.dataset.generate_ddi_mask_H_for_SafeDrug()
# MPNN_info = train_loader.dataset.dataset.generate_med_molecule_info_for_SafeDrug()
# ddi_adj = train_loader.dataset.dataset.get_ddi_matrix()
# model = RETAIN(
#     task = "drug_recommendation",
#     voc_size = drugrec_ds.voc_size,
#     tokenizers = drugrec_ds.tokenizers,
#     emb_dim = 64,
#     MPNN_info = MPNN_info,
#     bipartite_info = bipartite_info,
#     ddi_adj = ddi_adj,
# )

# ------ ML model -----------
model = MLModel(
    output_path="../output",
    task = "drug_recommendation",
#     classifier = SVC(gamma='auto', verbose=1),
#     classifier = LogisticRegression(random_state=0, max_iter=10, verbose=1),
#     classifier = RandomForestClassifier(random_state=0, n_estimators=20, max_depth=3),
    classifier = MLPClassifier(max_iter=50, alpha=1e-3, hidden_layer_sizes=(16, 16)),
    voc_size = drugrec_ds.voc_size,
    tokenizers = drugrec_ds.tokenizers
)

# 4. traning

In [23]:
# from pyhealth.trainer import Trainer
# from pyhealth.evaluator.evaluating_multilabel import evaluate_multilabel

# # # ----------- DL model ---------
# trainer = Trainer(enable_logging=True, output_path="../output")
# trainer.fit(model,
#             train_loader=train_loader,
#             epochs=50,
#             evaluate_fn=evaluate_multilabel,
#             eval_loader=val_loader,
#             monitor="jaccard")


# ------------ ML model -----------
model.fit(
    train_loader=train_loader,
    evaluate_fn=evaluate_multilabel,
    eval_loader=val_loader,
    monitor="jaccard",
)

(11680, 4706) (11680, 163)
best_model_path: ../output/221003-230717/best.ckpt


# 5. evaluation

In [24]:
# load the best model for ML model
# model.load(path="../output/221002-170055/best.ckpt")

# load the best model for DL model
# best_model = trainer.load(model, path="../output/221003-015604/best.ckpt") 

evaluate_multilabel(model, val_loader)

{'loss': 1.0,
 'ddi': 0.07266116297106122,
 'jaccard': 0.47675539329773964,
 'prauc': 0.738596427730712,
 'f1': 0.6366047688629042}

In [21]:
result = {'loss': 1.0,
 'ddi': 0.0776952846310419,
 'jaccard': 0.4483472284051399,
 'prauc': 0.7297839333122977,
 'f1': 0.6120128574119359}

print ("{:.4f} | {:.4f} | {:.4f} | {:.4f}".format(
    result['ddi'],
    result['jaccard'],
    result['prauc'],
    result['f1']
        )
      )

0.0777 | 0.4483 | 0.7298 | 0.6120
